In [1]:
dst_folder = '/Users/jd/Downloads/templates/' # папка для хранения файлов

url = 'http://loader.ri.eias.ru/Download.aspx?type=9&regcode=&guid=' # ссылка, в которую будем подставлять идентификатор файла

dst_file_name = 'tko_price_data.xlsx' # файл, в который собираем данные из файлов

template = 'FAS.JKH.OPEN.INFO.PRICE.TKO'
#template = 'FAS.JKH.OPEN.INFO.BALANCE.TKO'

In [10]:
%%time
import pandas as pd

# загружаем из файла список идентификаторов файлов
data = pd.read_excel('tko_files_list.xlsx', usecols = ['GUID', 'TEMPLATE', 'ADATE'] )
guids = data['GUID']

print(len(guids))
data.head()

7694
CPU times: user 549 ms, sys: 4.29 ms, total: 553 ms
Wall time: 559 ms


,TEMPLATE,ADATE,GUID
0,FAS.JKH.OPEN.INFO.ORG.TKO,2018-10-05 15:03:32,674f7faa-1e16-4ab6-b00e-8ded292c0895
1,FAS.JKH.OPEN.INFO.ORG.TKO,2018-10-08 11:07:19,f5abe406-7c4c-45c8-859f-f5687e46de61
2,FAS.JKH.OPEN.INFO.ORG.TKO,2018-10-08 14:30:23,48fa232c-ab25-491e-866d-6e0e36163f2b
3,FAS.JKH.OPEN.INFO.ORG.TKO,2018-10-08 15:17:50,96955737-e5a3-497e-9987-c45a3440aae4
4,FAS.JKH.OPEN.INFO.ORG.TKO,2018-10-09 09:55:00,e6195742-8263-4300-8285-743321949000


In [11]:
%%time
data = data[data['TEMPLATE'].str.contains(template)]
guids = data['GUID']
print(len(guids))
data.head()

2626
CPU times: user 7.02 ms, sys: 1.12 ms, total: 8.15 ms
Wall time: 7.32 ms


,TEMPLATE,ADATE,GUID
266,FAS.JKH.OPEN.INFO.PRICE.TKO,2018-11-26 14:18:16,b8b6fa08-47bd-4065-b1e1-17ed7a063ea4
268,FAS.JKH.OPEN.INFO.PRICE.TKO,2018-11-26 14:45:59,7b90dfa1-01cb-25ed-e053-8d78a8c031d5
270,FAS.JKH.OPEN.INFO.PRICE.TKO,2018-11-26 15:24:10,39f916fc-072e-436e-95ab-d359b2028903
271,FAS.JKH.OPEN.INFO.PRICE.TKO,2018-11-26 15:27:47,ab6b0ca1-6f28-45e9-b8dc-a296f08a09ae
279,FAS.JKH.OPEN.INFO.PRICE.TKO,2018-11-27 16:23:19,8afe6ce7-53a1-4482-be34-905acdec4c9a


In [16]:
%%time
# Скачиваем файлы
import os
import urllib.request
import sys
import datetime

folder_name = dst_folder+template+'/'
files = os.listdir(folder_name)

for file_guid in guids[:10]:
    if ( files.count(file_guid+".zip") == 0 ) :
        try:
            urllib.request.urlretrieve(url+file_guid, folder_name+file_guid+'.zip')
            #print(str(datetime.datetime.now())+" "+file_guid+" downloaded")
        except:
            print("Error with file_guid:"+file_guid, sys.exc_info()[0])

2020-08-19 10:03:37.447192 36774353-064d-4591-b413-0113649bbff9 downloaded
2020-08-19 10:03:38.603738 3f525f04-bb54-48cd-9f26-2e15109e1c9c downloaded
2020-08-19 10:03:39.877486 868bbee0-b3fc-d2a2-e053-8d78a8c031d5 downloaded
2020-08-19 10:03:41.147277 6470f908-a6cb-4992-bed2-e35bf57b174d downloaded
2020-08-19 10:03:42.596428 a0e2a350-b2cc-4cd8-85e9-72d932ce10ef downloaded
2020-08-19 10:03:43.911922 868c957b-2ea7-a529-e053-8d78a8c0e386 downloaded
2020-08-19 10:03:45.446276 2d852b16-9c51-498e-8db9-ddb4bd773849 downloaded
2020-08-19 10:03:46.708357 51758e39-e7cf-47de-92ff-79872197935b downloaded
2020-08-19 10:03:48.373176 fd91abf0-4d2d-49dc-afe0-077553182a4d downloaded
2020-08-19 10:03:49.820737 ac3f853b-8a3e-4db1-80a6-524031000c67 downloaded
CPU times: user 226 ms, sys: 222 ms, total: 448 ms
Wall time: 13.7 s


In [12]:
# специфическое разархивирование, чтобы на MacOS файлы разархивировались с читаемыми названиями
import os
import shutil
import zipfile
import re

def unpack_zipfile(filename, extract_dir, file_guid, encoding='cp437'):
    with zipfile.ZipFile(filename) as archive:
        for entry in archive.infolist():
            
            name = entry.filename.encode('cp437').decode(encoding)  # reencode!!!
            name = file_guid +' '+ re.sub(r'[^\w\s.]', '', str(name))
            
            # don't extract absolute paths or ones with .. in them
            if name.startswith('/') or '..' in name:
                print("Error with filenames "+filename)
                continue

            target = os.path.join(extract_dir, *name.split('/'))
            os.makedirs(os.path.dirname(target), exist_ok=True)
            if not entry.is_dir():  # file
                with archive.open(entry) as source, open(target, 'wb') as dest:
                    shutil.copyfileobj(source, dest)
    archive.close()

In [13]:
%%time
# Разархивируем файлы
import sys

folder_name = dst_folder+template+'/'
files = os.listdir(folder_name+'unzpd/')

for file_guid in guids:
    if ( not(any(file_guid in s for s in files)) ) :
        try:
            unpack_zipfile(folder_name+file_guid+'.zip', folder_name+'unzpd/', file_guid, encoding='cp866')
        except:
            print("Error with file_guid:"+file_guid, sys.exc_info()[0])



Error with filenames /Users/jd/Downloads/templates/FAS.JKH.OPEN.INFO.PRICE.TKO/bb6f78c1-7d5d-4cc0-98f0-922f794778d0.zip
Error with filenames /Users/jd/Downloads/templates/FAS.JKH.OPEN.INFO.PRICE.TKO/bb6f78c1-7d5d-4cc0-98f0-922f794778d0.zip
Error with filenames /Users/jd/Downloads/templates/FAS.JKH.OPEN.INFO.PRICE.TKO/bb6f78c1-7d5d-4cc0-98f0-922f794778d0.zip
Error with filenames /Users/jd/Downloads/templates/FAS.JKH.OPEN.INFO.PRICE.TKO/d51b5b5b-03da-4d2d-897c-bb5c7a919fef.zip
Error with filenames /Users/jd/Downloads/templates/FAS.JKH.OPEN.INFO.PRICE.TKO/d51b5b5b-03da-4d2d-897c-bb5c7a919fef.zip
Error with filenames /Users/jd/Downloads/templates/FAS.JKH.OPEN.INFO.PRICE.TKO/d51b5b5b-03da-4d2d-897c-bb5c7a919fef.zip
Error with filenames /Users/jd/Downloads/templates/FAS.JKH.OPEN.INFO.PRICE.TKO/c49732a9-41f6-4455-b5a6-858b26dba3f9.zip
Error with filenames /Users/jd/Downloads/templates/FAS.JKH.OPEN.INFO.PRICE.TKO/c49732a9-41f6-4455-b5a6-858b26dba3f9.zip
Error with filenames /Users/jd/Downloads

## Парсинг данных из файлов

### FAS.JKH.OPEN.INFO.PRICE.TKO

### метаданные файлов

In [14]:
%%time
import os, sys
from pyxlsb import open_workbook
import pandas as pd
import xml.etree.ElementTree
import xml.etree.cElementTree as ET
import zipfile
from datetime import datetime

template = 'FAS.JKH.OPEN.INFO.PRICE.TKO'
dst_file_name = 'tko_price_list.xlsx' # файл, в который собираем данные из файлов
dst_folder = '/Users/jd/Downloads/templates/' # папка для хранения файлов

cols=['file_guid','file_name','file_num','region','period','org','rst_org_id','modified']

folder_name = dst_folder+template+'/'
files = os.listdir(folder_name+'unzpd/')
df = pd.DataFrame(columns=cols)
file_num = 0

for filename in files:
    file_num = file_num + 1
    (region,period,org,rst_org_id,modified) = ('','','','','')
    if filename.endswith('.xlsb'):
        try:
            wb = open_workbook(folder_name+'unzpd/'+filename)
            ws = wb.get_sheet('Титульный')
            for row in ws.rows(sparse=False):
                if (row[5].r == 0): rst_org_id = int(row[5].v)
                if (row[4].v == 'Субъект РФ'): region = " ".join(str(row[5].v).split()).replace('None', '')
                if (row[4].v == 'Наименование организации'): org = " ".join(str(row[5].v).split()).replace('None', '')
                if (" ".join(str(row[4].v).split()) == 'Начало периода регулирования'): reg_period_start = " ".join(str(row[5].v).split()).replace('None', '')
                if (" ".join(str(row[4].v).split()) == 'Окончание периода регулирования'): reg_period_end = " ".join(str(row[5].v).split()).replace('None', '')
            period = reg_period_start + '-' + reg_period_end
                    
        except:
            print("Error with file: "+filename, sys.exc_info())

        try:
            zf = zipfile.ZipFile(folder_name+'unzpd/'+filename, 'r')
            part = zf.open('docProps/core.xml', 'r')
            tree = ET.XML(part.read())
            modified = tree.find('{http://purl.org/dc/terms/}modified').text
        except:
            print("Error with file: "+filename, sys.exc_info())

        df = df.append({'file_guid' : filename[0:36], 'file_name': filename, 'file_num': file_num,
                        'region': region, 'period': period, 'org': org, 'rst_org_id': rst_org_id,
                        'modified': datetime.fromisoformat(modified.replace('Z','')).strftime('%Y-%m-%d %H:%M:%S')
                       }, ignore_index=True)

print(df.info())
print(len(df))
df.to_excel(dst_file_name)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5298 entries, 0 to 5297
Data columns (total 8 columns):
file_guid     5298 non-null object
file_name     5298 non-null object
file_num      5298 non-null object
region        5298 non-null object
period        5298 non-null object
org           5298 non-null object
rst_org_id    5298 non-null object
modified      5298 non-null object
dtypes: object(8)
memory usage: 331.2+ KB
None
5298
CPU times: user 2min 7s, sys: 8.29 s, total: 2min 16s
Wall time: 2min 27s


In [16]:
# убрать лишние файлы
import pandas as pd
import os, sys

file_list = pd.read_excel('tko_price_list.xlsx', usecols = ['file_guid','file_name','file_num','region','period','org','rst_org_id','modified'] )
print(len(file_list))

file_list = file_list.sort_values(by=['modified']).groupby(['rst_org_id','region','period'], as_index=False).first()
print(len(file_list))

template = 'FAS.JKH.OPEN.INFO.PRICE.TKO'
dst_folder = '/Users/jd/Downloads/templates/' # папка для хранения файлов

folder_name = dst_folder+template+'/'
files = os.listdir(folder_name+'unzpd/')

for filename in list(set(files) - set(file_list['file_name'].to_list())):
    os.remove(folder_name+'unzpd/'+filename)

5298
1503


### данные

In [17]:
%%time
import os, sys
from pyxlsb import open_workbook
import pandas as pd

file_list = pd.read_excel('tko_price_list.xlsx', usecols = ['file_guid','file_name','file_num','region','period','org','rst_org_id','modified'] )
print(len(file_list))
# оставляем только файлы с максимальной датой изменения
file_list = file_list.sort_values(by=['modified']).groupby(['rst_org_id','region','period'], as_index=False).first()
print(len(file_list))

template = 'FAS.JKH.OPEN.INFO.PRICE.TKO'
dst_file_name = 'tko_price_data.xlsx' # файл, в который собираем данные из файлов
dst_folder = '/Users/jd/Downloads/templates/' # папка для хранения файлов

cols=['file_guid', 'file_name', 'file_num', 'sheet_name',
      'region','reg_period_start','reg_period_end','tr_date','tr_num','tr_corr_date','tr_corr_num',
      'org','inn','kpp','rst_org_id','mr','mo','oktmo','vdet',
      'nomer','tar_type','unit','tech_osob','ter','tko_type','tko_class',
      'tar_value','volume','date_begin','date_end'
     ]

folder_name = dst_folder+template+'/'
#files = os.listdir(folder_name+'unzpd/')
files = file_list['file_name'].to_list()
#collected_data = pd.read_excel(dst_file_name, usecols = cols )
#print(len(collected_data))

df = pd.DataFrame(columns=cols)
df0 = pd.DataFrame(columns=cols)
collected_data = pd.DataFrame(columns=cols)
file_num = 0

#for filename in ['9ae6c44e-2e5a-8d65-e053-8d78a8c06fab 001_FAS.JKH.OPEN.INFO.PRICE.TKO_Государственный комитет Республики Татарстан по тарифам REK.xlsb']:
for filename in files:
    file_num = file_num + 1
    if filename.endswith('.xlsb'):
        if ( len(collected_data[collected_data['file_guid'].str.contains(filename[0:36])]) == 0 ) :
            try:
                
                wb = open_workbook(folder_name+'unzpd/'+filename)
                
                tar_cnt = 0
                
                (sheet_name,region,reg_period_start,reg_period_end,tr_date,tr_num,tr_corr_date,tr_corr_num,
                org,inn,kpp,rst_org_id,mr,mo,oktmo,vdet,
                nomer,tar_type,unit,tech_osob,ter,tko_type,tko_class,
                tar_value,volume,date_begin,date_end) = ('','','','','','','','','','','','','','','','','','','','','','','','','','','')

                ws = wb.get_sheet('Титульный')

                for row in ws.rows(sparse=False):
                    if (row[5].r == 0): rst_org_id = int(row[5].v)
                    if (row[4].v == 'Субъект РФ'): region = " ".join(str(row[5].v).split()).replace('None', '')
                    if (" ".join(str(row[4].v).split()) == 'Начало периода регулирования'): reg_period_start = " ".join(str(row[5].v).split()).replace('None', '')
                    if (" ".join(str(row[4].v).split()) == 'Окончание периода регулирования'): reg_period_end = " ".join(str(row[5].v).split()).replace('None', '')
                    if (row[4].v == 'Дата документа об утверждении тарифов'): tr_date = " ".join(str(row[5].v).split()).replace('None', '')
                    if (row[4].v == 'Номер документа об утверждении тарифов'): tr_num = " ".join(str(row[5].v).split()).replace('None', '')
                    if (row[4].v == 'Дата принятия решения об изменении тарифов'): tr_corr_date = " ".join(str(row[5].v).split()).replace('None', '')
                    if (row[4].v == 'Номер принятия решения об изменении тарифов'): tr_corr_num = " ".join(str(row[5].v).split()).replace('None', '')
                    if (row[4].v == 'Наименование организации'): org = " ".join(str(row[5].v).split()).replace('None', '')
                    if (row[4].v == 'ИНН'): inn = " ".join(str(row[5].v).split()).replace('None', '')
                    if (row[4].v == 'КПП'): kpp = " ".join(str(row[5].v).split()).replace('None', '')
                                        
#                 if (tr_corr_date != ''): # Если указано решение об изменении, то учитываем его дату и номер
#                     tr_date = tr_corr_date
#                     tr_num = tr_corr_num
                
                sheetname = 'Форма 5.3.2'
                ws = wb.get_sheet(sheetname)
            
                df0 = pd.DataFrame(columns=cols)
                
                for row in ws.rows(sparse=False):
                    if (row[0].r >= 10):
                        r = ''
                        for cell in row:
                            r = r + str(cell.v)
                        
                        if (row[4].v == 'наименование тарифа'):
                            unit = " ".join(str(row[6].v).split()).replace('None', '')
                            nomer = " ".join(str(row[3].v).split()).replace('None', '')
                            for i in range(12):
                                if (len(row) >= 11+i*5):
                                    if ( (row[11+i*5].v != '')&(row[8+i*5].v != '')&(row[10+i*5].v != '') ):
                                        tar_value = float(" ".join(str(row[11+i*5].v).split()).replace('None', ''))
                                        date_begin = " ".join(str(row[8+i*5].v).split()).replace('None', '')
                                        date_end = " ".join(str(row[10+i*5].v).split()).replace('None', '')
                                        df0 = df0.append({
                                            'file_guid' : filename[0:36], 'file_name': filename, 'file_num': file_num, 'sheet_name': sheetname,
                                            'region': region, 'reg_period_start': reg_period_start, 'reg_period_end': reg_period_end,
                                            'tr_date': tr_date, 'tr_num': tr_num, 'tr_corr_date': tr_corr_date, 'tr_corr_num': tr_corr_num,
                                            'org': org, 'inn': inn, 'kpp': kpp, 'rst_org_id': rst_org_id,
                                            'nomer': nomer[:-2], 'unit': unit,
                                            'tar_value': tar_value, 'date_begin': date_begin, 'date_end': date_end
                                        }, ignore_index=True)
                            
                        if (r.find("Указывается наименование вида тарифа") >= 0):
                            tar_type = " ".join(str(row[4].v).split()).replace('None', '')
                        if (row[4].v == 'технологическая особенность'):
                            tech_osob = " ".join(str(row[5].v).split()).replace('None', '')
                        if (row[4].v == 'территория оказания услуг'):
                            ter = " ".join(str(row[5].v).split()).replace('None', '')
                        if (row[4].v == 'вид твердых коммунальных отходов'):
                            tko_type = " ".join(str(row[5].v).split()).replace('None', '')
                        if (row[4].v == 'класс опасности твердых коммунальных отходов '):
                            tko_class = " ".join(str(row[5].v).split()).replace('None', '')
                            df0['tar_type'] = tar_type
                            df0['tech_osob'] = tech_osob
                            df0['ter'] = ter
                            df0['tko_type'] = tko_type
                            df0['tko_class'] = tko_class
                            df = df.append(df0, ignore_index=True)
                            df0 = pd.DataFrame(columns=cols)
                            tar_cnt = tar_cnt+1


                sheetname = 'Форма 5.3.1'
                ws = wb.get_sheet(sheetname)

                df0 = pd.DataFrame(columns=cols)
                
                for row in ws.rows(sparse=False):
                    if (row[1].r >= 16):
                        r = ''
                        for cell in row:
                            r = r + str(cell.v)

                        if (row[4].v == 'наименование тарифа'):
                            unit = " ".join(str(row[6].v).split()).replace('None', '')
                            nomer = " ".join(str(row[3].v).split()).replace('None', '')
                            
                            if ( nomer[:2] == "2." ):
                                for i in range(12):
                                    if (len(row) > 11+i*6):
                                        if ( (row[11+i*6].v != '')&(row[8+i*6].v != '')&(row[10+i*6].v != '') ):
                                            try:
                                                volume = float(" ".join(str(row[11+i*6].v).split()).replace('None', ''))
                                            except:
                                                volume = 0
                                            date_begin = " ".join(str(row[8+i*6].v).split()).replace('None', '')
                                            date_end = " ".join(str(row[10+i*6].v).split()).replace('None', '')
                                            df0 = df0.append({
                                                'file_guid' : filename[0:36], 'file_name': filename, 'file_num': file_num, 'sheet_name': sheetname,
                                                'region': region, 'reg_period_start': reg_period_start, 'reg_period_end': reg_period_end,
                                                'tr_date': tr_date, 'tr_num': tr_num, 'tr_corr_date': tr_corr_date, 'tr_corr_num': tr_corr_num,
                                                'org': org, 'inn': inn, 'kpp': kpp, 'rst_org_id': rst_org_id,
                                                'nomer': nomer[2:-2], 'unit': unit,
                                                'volume': volume, 'date_begin': date_begin, 'date_end': date_end
                                            }, ignore_index=True)
                        if (r.find("Указывается наименование вида тарифа") >= 0):
                            tar_type = " ".join(str(row[4].v).split()).replace('None', '')
                        if (row[4].v == 'технологическая особенность'):
                            tech_osob = " ".join(str(row[5].v).split()).replace('None', '')
                        if (row[4].v == 'территория оказания услуг'):
                            ter = " ".join(str(row[5].v).split()).replace('None', '')
                        if (row[4].v == 'вид твердых коммунальных отходов'):
                            tko_type = " ".join(str(row[5].v).split()).replace('None', '')
                        if (row[4].v == 'класс опасности твердых коммунальных отходов '):
                            tko_class = " ".join(str(row[5].v).split()).replace('None', '')
                            df0['tar_type'] = tar_type
                            df0['tech_osob'] = tech_osob
                            df0['ter'] = ter
                            df0['tko_type'] = tko_type
                            df0['tko_class'] = tko_class
                            df = df.append(df0, ignore_index=True)
                            df0 = pd.DataFrame(columns=cols)
                            tar_cnt = tar_cnt+1
                
            except:
                print("Error with file: "+filename+", sheet "+sheetname, sys.exc_info())
                df = df.append({'file_guid' : filename[0:36], 'file_name': filename, 'file_num': file_num}, ignore_index=True)
    
    else:
        df = df.append({'file_guid' : filename[0:36], 'file_name': filename, 'file_num': file_num}, ignore_index=True)
        
    #os.remove(folder_name+'unzpd/'+filename)

collected_data = collected_data.append(df, ignore_index=True)
collected_data = collected_data.drop_duplicates(subset = cols)


print(collected_data.info())
print(len(collected_data))


collected_data.to_excel(dst_file_name)

5298
1503
<class 'pandas.core.frame.DataFrame'>
Int64Index: 14129 entries, 0 to 14128
Data columns (total 30 columns):
file_guid           14129 non-null object
file_name           14129 non-null object
file_num            14129 non-null object
sheet_name          14129 non-null object
region              14129 non-null object
reg_period_start    14129 non-null object
reg_period_end      14129 non-null object
tr_date             14129 non-null object
tr_num              14129 non-null object
tr_corr_date        14129 non-null object
tr_corr_num         14129 non-null object
org                 14129 non-null object
inn                 14129 non-null object
kpp                 14129 non-null object
rst_org_id          14129 non-null object
mr                  0 non-null object
mo                  0 non-null object
oktmo               0 non-null object
vdet                0 non-null object
nomer               14129 non-null object
tar_type            14129 non-null object
unit           

### FAS.JKH.OPEN.INFO.BALANCE.TKO

### метаданные файлов

In [6]:
%%time
import os, sys
from pyxlsb import open_workbook
import pandas as pd
import xml.etree.ElementTree
import xml.etree.cElementTree as ET
import zipfile
from datetime import datetime

template = 'FAS.JKH.OPEN.INFO.BALANCE.TKO'
dst_file_name = 'tko_balance_list.xlsx' # файл, в который собираем данные из файлов
dst_folder = '/Users/jd/Downloads/templates/' # папка для хранения файлов

cols=['file_guid','file_name','file_num','region','period','org','rst_org_id','modified']

folder_name = dst_folder+template+'/'
files = os.listdir(folder_name+'unzpd/')
df = pd.DataFrame(columns=cols)
file_num = 0

for filename in files:
    file_num = file_num + 1
    (region,period,org,rst_org_id,modified) = ('','','','','')
    if filename.endswith('.xlsb'):
        try:
            wb = open_workbook(folder_name+'unzpd/'+filename)
            ws = wb.get_sheet('Титульный')
            for row in ws.rows(sparse=False):
                if (row[5].r == 0): rst_org_id = int(row[5].v)
                if (row[4].v == 'Субъект РФ'): region = " ".join(str(row[5].v).split()).replace('None', '')
                if (row[4].v == 'Отчётный период (год)'): period = int(float(" ".join(str(row[5].v).split()).replace('None', '')))
                if (row[4].v == 'Наименование организации'): org = " ".join(str(row[5].v).split()).replace('None', '')
        except:
            print("Error with file: "+filename, sys.exc_info())

        try:
            zf = zipfile.ZipFile(folder_name+'unzpd/'+filename, 'r')
            part = zf.open('docProps/core.xml', 'r')
            tree = ET.XML(part.read())
            modified = tree.find('{http://purl.org/dc/terms/}modified').text
        except:
            print("Error with file: "+filename, sys.exc_info())

        df = df.append({'file_guid' : filename[0:36], 'file_name': filename, 'file_num': file_num,
                        'region': region, 'period': period, 'org': org, 'rst_org_id': rst_org_id,
                        'modified': datetime.fromisoformat(modified.replace('Z','')).strftime('%Y-%m-%d %H:%M:%S')
                       }, ignore_index=True)

print(df.info())
print(len(df))
df.to_excel(dst_file_name)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4314 entries, 0 to 4313
Data columns (total 8 columns):
file_guid     4314 non-null object
file_name     4314 non-null object
file_num      4314 non-null object
region        4314 non-null object
period        4314 non-null object
org           4314 non-null object
rst_org_id    4314 non-null object
modified      4314 non-null object
dtypes: object(8)
memory usage: 269.8+ KB
None
4314
CPU times: user 1min 45s, sys: 7.66 s, total: 1min 52s
Wall time: 2min 3s


In [7]:
# убрать лишние файлы
import pandas as pd
import os, sys

file_list = pd.read_excel('tko_balance_list.xlsx', usecols = ['file_guid','file_name','file_num','region','period','org','rst_org_id','modified'] )
print(len(file_list))

file_list = file_list.sort_values(by=['modified']).groupby(['rst_org_id','region','period'], as_index=False).first()

print(len(file_list))

template = 'FAS.JKH.OPEN.INFO.BALANCE.TKO'
dst_folder = '/Users/jd/Downloads/templates/' # папка для хранения файлов

folder_name = dst_folder+template+'/'
files = os.listdir(folder_name+'unzpd/')

for filename in list(set(files) - set(file_list['file_name'].to_list())):
    os.remove(folder_name+'unzpd/'+filename)



4314
1229


### данные

In [8]:
%%time
import os, sys
from pyxlsb import open_workbook
import pandas as pd

file_list = pd.read_excel(dst_file_name, usecols = cols )
file_list = pd.read_excel('tko_balance_list.xlsx', usecols = ['file_guid','file_name','file_num','region','period','org','rst_org_id','modified'] )
print(len(file_list))
# оставляем только файлы с максимальной датой изменения
file_list = file_list.sort_values(by=['modified']).groupby(['rst_org_id','region','period'], as_index=False).first()
print(len(file_list))

template = 'FAS.JKH.OPEN.INFO.BALANCE.TKO'
dst_file_name = 'tko_balance_data.xlsx' # файл, в который собираем данные из файлов
dst_folder = '/Users/jd/Downloads/templates/' # папка для хранения файлов

cols=['file_guid', 'file_name', 'file_num', 'sheet_name',
      'region','report_type','period','org','inn','kpp','rst_org_id','mr','mo','oktmo','vdet',
      'unit','L2','L3','L4','L5','L7','L8','L9',
      'L3_1','L3_1_1','L3_1_2','L3_2','L3_2_1','L3_2_2','L3_3','L3_3_1','L3_3_2','L3_4','L3_4_1','L3_4_2',
      'L3_5','L3_6','L3_7','L3_8',
      'buh_link'
     ]

folder_name = dst_folder+template+'/'
#files = os.listdir(folder_name+'unzpd/')
files = file_list['file_name'].to_list()
#collected_data = pd.read_excel(dst_file_name, usecols = cols )
#print(len(collected_data))

df = pd.DataFrame(columns=cols)
collected_data = pd.DataFrame(columns=cols)
file_num = 0

for filename in files:
    file_num = file_num + 1
    if filename.endswith('.xlsb'):
        if ( len(collected_data[collected_data['file_guid'].str.contains(filename[0:36])]) == 0 ) :
            try:
                
                wb = open_workbook(folder_name+'unzpd/'+filename)
                
                (sheet_name,region,period,org,inn,kpp,rst_org_id,mr,mo,oktmo,vdet,unit,buh_link,
                 L1,L2,L3,L4,L5,L7,L8,L9,
                 L3_1,L3_1_1,L3_1_2,L3_2,L3_2_1,L3_2_2,L3_3,L3_3_1,L3_3_2,L3_4,L3_4_1,L3_4_2,
                 L3_5,L3_6,L3_7,L3_8) = ('','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','')

                ws = wb.get_sheet('Титульный')

                for row in ws.rows(sparse=False):
                    if (row[5].r == 0): rst_org_id = int(row[5].v)
                    if (row[4].v == 'Субъект РФ'): region = " ".join(str(row[5].v).split()).replace('None', '')
                    if (row[4].v == 'Тип отчета'): report_type = " ".join(str(row[5].v).split()).replace('None', '')
                    if (row[4].v == 'Отчётный период (год)'): period = int(float(" ".join(str(row[5].v).split()).replace('None', '')))
                    if (row[4].v == 'Наименование организации'): org = " ".join(str(row[5].v).split()).replace('None', '')
                    if (row[4].v == 'ИНН'): inn = " ".join(str(row[5].v).split()).replace('None', '')
                    if (row[4].v == 'КПП'): kpp = " ".join(str(row[5].v).split()).replace('None', '')

                sheetname = 'Форма 5.4.1'
                ws = wb.get_sheet(sheetname)

                for row in ws.rows(sparse=False):
                    if (row[3].v == '№ п/п'): vdet = " ".join(str(row[6].v).split()).replace('None', '')
                    if (row[3].v == '2'):
                        unit = " ".join(str(row[5].v).split()).replace('None', '')
                        L2 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '2'): L3 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.1'): L3_1 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.1.1'): L3_1_1 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.1.2'): L3_1_2 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.2'): L3_2 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.2.1'): L3_2_1 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.2.2'): L3_2_2 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.3'): L3_3 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.3.1'): L3_3_1 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.3.2'): L3_3_2 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.4'): L3_4 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.4.1'): L3_4_1 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.4.2'): L3_4_2 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.5'): L3_5 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.6'): L3_6 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.7'): L3_7 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '3.8'): L3_8 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '4'): L4 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '5'): L5 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '7'): L7 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '8'): L8 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '9'): L9 = round(float(" ".join(str(row[6].v).split()).replace('None', '')),4)
                    if (row[3].v == '6'): buh_link = " ".join(str(row[6].v).split()).replace('None', '')
                    
                df = df.append({
                    'file_guid' : filename[0:36], 'file_name': filename, 'file_num': file_num, 'sheet_name': sheetname,
                    'region': region, 'report_type': report_type, 'period': period,
                    'mr': mr, 'mo': mo, 'oktmo': oktmo, 'vdet': vdet,
                    'unit': unit, 'org': org, 'inn': inn, 'kpp': kpp, 'rst_org_id': rst_org_id,
                    'L2': L2, 'L3': L3, 'L4': L4, 'L5': L5, 'L7': L7, 'L8': L8, 'L9': L9,
                    'L3_1': L3_1, 'L3_1_1': L3_1_1, 'L3_1_2': L3_1_2,
                    'L3_2': L3_2, 'L3_2_1': L3_2_1, 'L3_2_2': L3_2_2,
                    'L3_3': L3_3, 'L3_3_1': L3_3_1, 'L3_3_2': L3_3_2,
                    'L3_4': L3_4, 'L3_4_1': L3_4_1, 'L3_4_2': L3_4_2,
                    'L3_5': L3_5, 'L3_6': L3_6, 'L3_7': L3_7, 'L3_8': L3_8,
                    'buh_link': buh_link
                }, ignore_index=True)
                #print(df)
                
            except:
                print("Error with file: "+filename, sys.exc_info())
                df = df.append({'file_guid' : filename[0:36], 'file_name': filename, 'file_num': file_num}, ignore_index=True)
    
    else:
        df = df.append({'file_guid' : filename[0:36], 'file_name': filename, 'file_num': file_num}, ignore_index=True)
        
    #os.remove(folder_name+'unzpd/'+filename)

collected_data = collected_data.append(df, ignore_index=True)
collected_data = collected_data.drop_duplicates(subset = cols)


print(collected_data.info())
print(len(collected_data))


collected_data.to_excel(dst_file_name)

# 1000 guids - 4 min4

4314
Error with file: a8ad08bc-7725-c782-e053-8d78a8c0a6fa 001_FAS.JKH.OPEN.INFO.BALANCE.TKO_Агентство по регулированию цен и тарифов Ульяновской области REK.xlsb (<class 'ValueError'>, ValueError('could not convert string to float: '), <traceback object at 0x7fee70e8e460>)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1229 entries, 0 to 1228
Data columns (total 40 columns):
file_guid      1229 non-null object
file_name      1229 non-null object
file_num       1229 non-null object
sheet_name     1228 non-null object
region         1228 non-null object
report_type    1228 non-null object
period         1228 non-null object
org            1228 non-null object
inn            1228 non-null object
kpp            1228 non-null object
rst_org_id     1228 non-null object
mr             1228 non-null object
mo             1228 non-null object
oktmo          1228 non-null object
vdet           1228 non-null object
unit           1228 non-null object
L2             1228 non-null float64
L3   

In [7]:
collected_data

,file_guid,file_name,file_num,sheet_name,region,report_type,period,org,inn,kpp,...,L3_3_1,L3_3_2,L3_4,L3_4_1,L3_4_2,L3_5,L3_6,L3_7,L3_8,buh_link
0,871fd35a-7b0b-7e81-e053-8d78a8c07a89,871fd35a-7b0b-7e81-e053-8d78a8c07a89 001_FAS.J...,852,Форма 5.4.1,Республика Крым,изменения в раскрытой ранее информации,2017.0,"МУП ""Экоград"" г. Евпатория",9.110088e+09,911001001,...,474.990,141.040,531.3700,531.3700,0.0000,1125.9800,0.0,0.0,772.1515,NaN
1,a40003c6-6017-4a3a-bcb2-7e279c41c94c,a40003c6-6017-4a3a-bcb2-7e279c41c94c 002_FAS.J...,766,Форма 5.4.1,Саратовская область,первичное раскрытие информации,2018.0,"АО ""Управление отходами""",7.725727e+09,645043001,...,10387.106,2994.438,258853.6390,397.5160,258456.1230,5814.7750,0.0,0.0,255011.5630,https://portal.eias.ru/Portal/DownloadPage.asp...
2,2c25fe57-dbfd-4d0b-a183-745c89ac652b,2c25fe57-dbfd-4d0b-a183-745c89ac652b 002_FAS.J...,916,Форма 5.4.1,Ивановская область,первичное раскрытие информации,2018.0,"ООО ""Региональный оператор по обращению с ТКО""",3.704006e+09,370201001,...,79368.000,20799.000,50034.0000,50034.0000,0.0000,7914.0000,8314.0,0.0,15418.0000,https://portal.eias.ru/Portal/DownloadPage.asp...
3,a7c695ca-ecd3-c2b0-e053-8d78a8c0fc64,a7c695ca-ecd3-c2b0-e053-8d78a8c0fc64 001_FAS.J...,814,Форма 5.4.1,Кемеровская область,первичное раскрытие информации,2018.0,"ООО ""ЭкоТек""",4.217127e+09,421701001,...,0.000,0.000,336.2566,238.7399,97.5167,8015.5376,0.0,0.0,0.0000,https://portal.eias.ru/Portal/DownloadPage.asp...
4,d052da4e-14f6-4614-828e-7f9299c17a4d,d052da4e-14f6-4614-828e-7f9299c17a4d 003_FAS.J...,1153,Форма 5.4.1,Республика Коми,первичное раскрытие информации,2018.0,"ООО ""УХТАЖИЛФОНД""",1.102055e+09,110201001,...,10209.470,3065.250,2219.5900,2219.5900,0.0000,9560.3700,0.0,0.0,170466.8400,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,a3cbeae7-4175-0c8d-e053-8a2b000a9a04,a3cbeae7-4175-0c8d-e053-8a2b000a9a04 001_FAS.J...,1228,Форма 5.4.1,Самарская область,первичное раскрытие информации,2019.0,АО Экотехнопарк «Зелененький»,6.316188e+09,631401001,...,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0000,https://portal.eias.ru/Portal/DownloadPage.asp...
1225,5a92cd29-c1a5-4b06-bab1-2185748bac26,5a92cd29-c1a5-4b06-bab1-2185748bac26 002_FAS.J...,46,Форма 5.4.1,Свердловская область,первичное раскрытие информации,2019.0,Муниципальное унитарное предприятие жилищно-ко...,6.652016e+09,668501001,...,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0000,https://portal.eias.ru/Portal/DownloadPage.asp...
1226,fcd38e1a-9d16-4c6d-9fdd-d9eae193c8e0,fcd38e1a-9d16-4c6d-9fdd-d9eae193c8e0 002_FAS.J...,569,Форма 5.4.1,Свердловская область,первичное раскрытие информации,2019.0,"Общество с ограниченной ответственностью ""Энер...",6.615013e+09,661501001,...,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0000,https://portal.eias.ru/Portal/DownloadPage.asp...
1227,2d3c0973-f9c4-4b17-93f3-cef06b9e4a75,2d3c0973-f9c4-4b17-93f3-cef06b9e4a75 002_FAS.J...,882,Форма 5.4.1,Чукотский автономный округ,первичное раскрытие информации,2019.0,"ООО ""АТК""",8.709013e+09,870901001,...,988.600,231.300,3081.4000,3081.4000,0.0000,162.0000,0.0,0.0,1405.2000,NaN
